Investigate why ingest_v2 got stuck in DocumentTopicFactory

In [1]:
from askem.elastic import DocumentTopicFactory

In [2]:
doc_topic_factory = DocumentTopicFactory(["climate-change-modeling"])
doc_topic_factory.run()

Getting ids for climate-change-modeling


climate-change-modeling: 100%|██████████| 136835/136835 [01:38<00:00, 1382.66ids/s]


Found 136835 ids for climate-change-modeling


{'5a2d846ccf58f1103e4024ac': ['climate-change-modeling'],
 '5a2db5c4cf58f12162d47c5b': ['climate-change-modeling'],
 '5a2dbca5cf58f125126df494': ['climate-change-modeling'],
 '5a049ddfcf58f1b96402bd43': ['climate-change-modeling'],
 '5a04d007cf58f155eb5963ad': ['climate-change-modeling'],
 '5a04d740cf58f15890c5528c': ['climate-change-modeling'],
 '5a04d7a6cf58f15890c552e4': ['climate-change-modeling'],
 '5a04f36acf58f163006af3e1': ['climate-change-modeling'],
 '5a050260cf58f16826bc0684': ['climate-change-modeling'],
 '5a2a4db9cf58f159a7d69f65': ['climate-change-modeling'],
 '5a2a7f11cf58f16c21cacedb': ['climate-change-modeling'],
 '5a2a85e5cf58f16ebeb51e29': ['climate-change-modeling'],
 '5a2a8ceecf58f1714fab03d1': ['climate-change-modeling'],
 '5a2a8cefcf58f1714fab03d3': ['climate-change-modeling'],
 '5a2a8cf5cf58f1714fab03db': ['climate-change-modeling'],
 '5a2a9403cf58f173d436e5ca': ['climate-change-modeling'],
 '5a2a9419cf58f173d436e630': ['climate-change-modeling'],
 '5a2aa919cf58